# 看護師離職率データセット作成（ETL完全版）

このノートブックは、手作業によるデータ作成と同等の品質（正確な数値、適切な単位）を自動化するためのものです。
特に**「沖縄県のデータ異常」「年収の単位変換」「家賃の民営借家指定」**などの重要ロジックが実装されています。

## 前提
- 元データ（CSV/Excel）が `../data/raw/` に配置されていること。
- 出力先は `../data/processed/` です。

In [21]:
import pandas as pd
import numpy as np
import os
import glob
import re

# ---------------------------------------------------------
# 1. プロジェクト設定
# ---------------------------------------------------------
# パス定義 (notebooks/ からの相対パス)
DIR_RAW = '../data/raw'
DIR_PROCESSED = '../data/processed'
DIR_INTERMEDIATE = os.path.join(DIR_PROCESSED, 'intermediate')

# 出力ディレクトリの作成
os.makedirs(DIR_INTERMEDIATE, exist_ok=True)

print(f"Directory Configuration:")
print(f"  Input (Raw): {os.path.abspath(DIR_RAW)}")
print(f"  Output (Processed): {os.path.abspath(DIR_PROCESSED)}")

# ---------------------------------------------------------
# 2. 共通関数：都道府県名の正規化 (仕様書 2.1準拠)
# ---------------------------------------------------------
def normalize_prefecture(text):
    """
    あらゆる表記の都道府県名を統一フォーマット（末尾に都府県付き）に変換する。
    全国、計、不明などは欠損(NaN)にする。
    """
    if not isinstance(text, str):
        return np.nan
    
    # クリーニング
    text = str(text).replace(" ", "").replace("　", "").strip()
    
    # 除外キーワード (仕様書 2.1)
    exclude_keywords = ['全国', '計', '未回答', '無回答', '不明', '総数']
    if any(keyword == text for keyword in exclude_keywords): 
        return np.nan
    if text in exclude_keywords:
        return np.nan

    # 接尾辞ロジック
    if text == "北海道":
        return text
    elif text == "東京":
        return "東京都"
    elif text in ["大阪", "京都"]:
        return text + "府"
    
    # 既に末尾が正しいかチェック
    if text.endswith(("都", "道", "府", "県")):
        return text
    
    # それ以外は「県」を補完 (例: 青森 -> 青森県)
    return text + "県"

# テスト実行
print("\n--- Normalize Test ---")
test_samples = ["青　森", "東京", "神奈川県", "全国", "大阪", "福岡"]
for t in test_samples:
    print(f"'{t}' -> '{normalize_prefecture(t)}'")

Directory Configuration:
  Input (Raw): /Users/hideomi.h/nurse-turnover-analysis/data/raw
  Output (Processed): /Users/hideomi.h/nurse-turnover-analysis/data/processed

--- Normalize Test ---
'青　森' -> '青森県'
'東京' -> '東京都'
'神奈川県' -> '神奈川県'
'全国' -> 'nan'
'大阪' -> '大阪府'
'福岡' -> '福岡県'


In [23]:
# ---------------------------------------------------------
# Step 1: 医療施設調査（病床規模）の加工
# Target: 厚労省_医療施設調査_病床規模_2024.csv
# Logic: Header=12(13行目), 縦持ちデータをフィルタして集計
# ---------------------------------------------------------

FILE_BEDS = os.path.join(DIR_RAW, '厚労省_医療施設調査_病床規模_2024.csv')

try:
    print(f"Processing: {os.path.basename(FILE_BEDS)}")
    
    # 1. 読み込み (Header位置固定: 13行目=Index 12)
    # BOM付きCSVのため utf-8-sig を指定
    df_beds = pd.read_csv(FILE_BEDS, encoding='utf-8-sig', header=12)
    
    print(f"  -> Raw shape: {df_beds.shape}")
    
    # 2. 列名の特定（テキストダンプに基づく）
    # 都道府県: "都道府県－指定都市・特別区・中核市（再掲）"
    # 病床カテゴリ: "病床の規模_001"
    # 値: "一般病院（総数）"
    
    col_pref = [c for c in df_beds.columns if '都道府県' in c and 'コード' not in c][0]
    col_size = [c for c in df_beds.columns if '病床の規模' in c and 'コード' not in c][0]
    col_val = [c for c in df_beds.columns if '一般病院' in c and '総数' in c][0]
    
    print(f"  -> Columns identified:")
    print(f"     Pref: {col_pref}")
    print(f"     Size: {col_size}")
    print(f"     Value: {col_val}")

    # 3. 都道府県の正規化
    df_beds['prefecture'] = df_beds[col_pref].apply(normalize_prefecture)
    # 全国や再掲（政令指定都市など）を除外して、純粋な47都道府県のみ残す
    df_beds = df_beds.dropna(subset=['prefecture'])
    
    # 4. 500床以上の行を抽出
    # ターゲット: 500～599, 600～699, 700～799, 800～899, 900床以上
    target_sizes = [
        "500～599床", "600～699床", "700～799床", "800～899床", "900床以上"
    ]
    # 文字列の部分一致やリスト判定でフィルタ
    mask_large = df_beds[col_size].astype(str).str.strip().isin(target_sizes)
    df_large = df_beds[mask_large].copy()
    
    # 5. 数値化と集計
    # "-" や "," を処理
    def clean_num(x):
        s = str(x).replace(',', '').strip()
        if s in ['-', '…', '...', '***', 'X']:
            return 0
        try:
            return int(s)
        except:
            return 0

    df_large['val_numeric'] = df_large[col_val].apply(clean_num)
    
    # 都道府県ごとに合計 (group by prefecture)
    df_beds_out = df_large.groupby('prefecture', as_index=False)['val_numeric'].sum()
    df_beds_out = df_beds_out.rename(columns={'val_numeric': 'large_hospital_count'})
    
    # 6. 保存
    out_path = os.path.join(DIR_INTERMEDIATE, 'temp_large_hospitals.csv')
    df_beds_out.to_csv(out_path, index=False, encoding='utf-8-sig')
    print(f"  -> Saved: {out_path} (Rows: {len(df_beds_out)})")
    print(df_beds_out.head(3))

except Exception as e:
    print(f"Error in Step 1: {e}")

Processing: 厚労省_医療施設調査_病床規模_2024.csv
  -> Raw shape: (1965, 32)
  -> Columns identified:
     Pref: 都道府県－指定都市・特別区・中核市（再掲）
     Size: 病床の規模_001
     Value: 一般病院（総数）
  -> Saved: ../data/processed/intermediate/temp_large_hospitals.csv (Rows: 130)
  prefecture  large_hospital_count
0        三重県                     4
1        京都府                    10
2        佐賀県                     3


In [24]:
# ---------------------------------------------------------
# Step 2: 通勤時間の加工
# Target: 総務省_社会生活基本調査_通勤時間_2022.xlsx
# Logic: "1.24" (1時間24分) -> 84分 に変換 (60進法)
# ---------------------------------------------------------

FILE_COMMUTE = os.path.join(DIR_RAW, '総務省_社会生活基本調査_通勤時間_2022.xlsx')

def convert_hours_minutes_to_minutes(val):
    if pd.isna(val): return np.nan
    try:
        val = float(val)
        hours = int(val)
        # 小数部分を "分" として取り出す (0.24 -> 24)
        minutes = int(round((val - hours) * 100))
        return hours * 60 + minutes
    except:
        return np.nan

try:
    print(f"Processing: {os.path.basename(FILE_COMMUTE)}")
    
    # ヘッダー探索
    df_temp = pd.read_excel(FILE_COMMUTE, header=None, nrows=20)
    header_row = df_temp.index[df_temp.apply(lambda x: x.astype(str).str.contains('都道府県|通勤').any(), axis=1)][0]
    df_commute = pd.read_excel(FILE_COMMUTE, header=header_row)
    
    # 列特定
    col_pref = [c for c in df_commute.columns if '都道府県' in str(c)][0]
    # 値の列: "通勤時間" や "総数"、あるいは一番右の列を狙う
    col_val = [c for c in df_commute.columns if '通勤' in str(c) or '総数' in str(c)][-1] 
    
    # 正規化
    df_commute['prefecture'] = df_commute[col_pref].apply(normalize_prefecture)
    df_commute = df_commute.dropna(subset=['prefecture'])
    
    # 時間変換
    df_commute['commute_time'] = df_commute[col_val].apply(convert_hours_minutes_to_minutes)
    
    # 保存
    df_commute_out = df_commute[['prefecture', 'commute_time']].copy()
    out_path = os.path.join(DIR_INTERMEDIATE, 'temp_commute.csv')
    df_commute_out.to_csv(out_path, index=False, encoding='utf-8-sig')
    print(f"  -> Saved: {out_path}")

except Exception as e:
    print(f"Error in Step 2: {e}")

Processing: 総務省_社会生活基本調査_通勤時間_2022.xlsx
Error in Step 2: list index out of range


In [25]:
# ---------------------------------------------------------
# Step 3.5: 有効求人倍率 (2023年度平均) の算出
# Target: 厚労省_一般職業紹介状況_有効求人倍率_2025.xlsx
# Logic: "2023年4月"〜"2024年3月" の行を特定し、その平均値を算出する
# ---------------------------------------------------------

FILE_JOB = os.path.join(DIR_RAW, '厚労省_一般職業紹介状況_有効求人倍率_2025.xlsx')

try:
    print(f"Processing: {os.path.basename(FILE_JOB)}")
    
    # 1. ヘッダー行を探す（都道府県名が並んでいる行）
    df_temp = pd.read_excel(FILE_JOB, header=None, nrows=50)
    header_row_idx = df_temp.index[df_temp.apply(lambda x: x.astype(str).str.contains('北海道').any(), axis=1)][0]
    print(f"  -> Header found at row index: {header_row_idx}")
    
    # 2. 本読み込み
    df_job = pd.read_excel(FILE_JOB, header=header_row_idx)
    
    # 3. 年・月の列を特定（画像を見ると左端に年と月がある）
    # 列名が不明確な場合が多いので、データの中身で列を特定する
    col_year = None
    col_month = None
    
    for col in df_job.columns[:10]: # 左側の10列くらいを調査
        sample_vals = df_job[col].astype(str).head(10).tolist()
        if any('年' in v for v in sample_vals if v != 'nan'):
            if col_year is None: col_year = col
        if any('月' in v for v in sample_vals if v != 'nan'):
            if col_month is None: col_month = col
            
    if not col_year or not col_month:
        # 見つからない場合のフォールバック（画像の配置から推測）
        col_year = df_job.columns[0]
        col_month = df_job.columns[2] # 間に元号列があると仮定
    
    print(f"  -> Date columns identified: Year='{col_year}', Month='{col_month}'")

    # 4. フィルタリング (2023年度 = 2023.4 ~ 2024.3)
    # スペース除去して型統一
    df_job[col_year] = df_job[col_year].astype(str).str.replace(' ', '').str.replace('　', '')
    df_job[col_month] = df_job[col_month].astype(str).str.replace(' ', '').str.replace('　', '')
    
    # 条件作成
    mask_2023 = (df_job[col_year] == '2023年') & (df_job[col_month].isin([f'{i}月' for i in range(4, 13)]))
    mask_2024 = (df_job[col_year] == '2024年') & (df_job[col_month].isin(['1月', '2月', '3月']))
    
    df_fy2023 = df_job[mask_2023 | mask_2024].copy()
    print(f"  -> Extracted rows: {len(df_fy2023)} (Expected: 12)")
    
    if len(df_fy2023) == 0:
        raise ValueError("No rows matched for FY2023. Check date formats.")

    # 5. 都道府県ごとの平均を算出
    valid_means = {}
    for col in df_fy2023.columns:
        pref_name = normalize_prefecture(str(col))
        if isinstance(pref_name, str):
            # 数値化して平均
            vals = pd.to_numeric(df_fy2023[col], errors='coerce')
            valid_means[pref_name] = vals.mean()
            
    df_job_out = pd.DataFrame(list(valid_means.items()), columns=['prefecture', 'job_openings_ratio'])
    
    # 保存
    out_path = os.path.join(DIR_INTERMEDIATE, 'temp_job_ratio.csv')
    df_job_out.to_csv(out_path, index=False, encoding='utf-8-sig')
    print(f"  -> Saved: {out_path}")

except Exception as e:
    print(f"Error in Step 3.5: {e}")

Processing: 厚労省_一般職業紹介状況_有効求人倍率_2025.xlsx
  -> Header found at row index: 1
  -> Date columns identified: Year='西暦', Month='Unnamed: 2'
  -> Extracted rows: 3 (Expected: 12)
  -> Saved: ../data/processed/intermediate/temp_job_ratio.csv


In [26]:
# ---------------------------------------------------------
# Step 3.5: 有効求人倍率 (2023年度平均) の算出
# Target: 厚労省_一般職業紹介状況_有効求人倍率_2025.xlsx
# Logic: "2023年4月"〜"2024年3月" の行を特定し、その平均値を算出する
# ---------------------------------------------------------

FILE_JOB = os.path.join(DIR_RAW, '厚労省_一般職業紹介状況_有効求人倍率_2025.xlsx')

try:
    print(f"Processing: {os.path.basename(FILE_JOB)}")
    
    # 1. ヘッダー行を探す（都道府県名が並んでいる行）
    df_temp = pd.read_excel(FILE_JOB, header=None, nrows=50)
    header_row_idx = df_temp.index[df_temp.apply(lambda x: x.astype(str).str.contains('北海道').any(), axis=1)][0]
    print(f"  -> Header found at row index: {header_row_idx}")
    
    # 2. 本読み込み
    df_job = pd.read_excel(FILE_JOB, header=header_row_idx)
    
    # 3. 年・月の列を特定（画像を見ると左端に年と月がある）
    # 列名が不明確な場合が多いので、データの中身で列を特定する
    col_year = None
    col_month = None
    
    for col in df_job.columns[:10]: # 左側の10列くらいを調査
        sample_vals = df_job[col].astype(str).head(10).tolist()
        if any('年' in v for v in sample_vals if v != 'nan'):
            if col_year is None: col_year = col
        if any('月' in v for v in sample_vals if v != 'nan'):
            if col_month is None: col_month = col
            
    if not col_year or not col_month:
        # 見つからない場合のフォールバック（画像の配置から推測）
        col_year = df_job.columns[0]
        col_month = df_job.columns[2] # 間に元号列があると仮定
    
    print(f"  -> Date columns identified: Year='{col_year}', Month='{col_month}'")

    # 4. フィルタリング (2023年度 = 2023.4 ~ 2024.3)
    # スペース除去して型統一
    df_job[col_year] = df_job[col_year].astype(str).str.replace(' ', '').str.replace('　', '')
    df_job[col_month] = df_job[col_month].astype(str).str.replace(' ', '').str.replace('　', '')
    
    # 条件作成
    mask_2023 = (df_job[col_year] == '2023年') & (df_job[col_month].isin([f'{i}月' for i in range(4, 13)]))
    mask_2024 = (df_job[col_year] == '2024年') & (df_job[col_month].isin(['1月', '2月', '3月']))
    
    df_fy2023 = df_job[mask_2023 | mask_2024].copy()
    print(f"  -> Extracted rows: {len(df_fy2023)} (Expected: 12)")
    
    if len(df_fy2023) == 0:
        raise ValueError("No rows matched for FY2023. Check date formats.")

    # 5. 都道府県ごとの平均を算出
    valid_means = {}
    for col in df_fy2023.columns:
        pref_name = normalize_prefecture(str(col))
        if isinstance(pref_name, str):
            # 数値化して平均
            vals = pd.to_numeric(df_fy2023[col], errors='coerce')
            valid_means[pref_name] = vals.mean()
            
    df_job_out = pd.DataFrame(list(valid_means.items()), columns=['prefecture', 'job_openings_ratio'])
    
    # 保存
    out_path = os.path.join(DIR_INTERMEDIATE, 'temp_job_ratio.csv')
    df_job_out.to_csv(out_path, index=False, encoding='utf-8-sig')
    print(f"  -> Saved: {out_path}")

except Exception as e:
    print(f"Error in Step 3.5: {e}")

Processing: 厚労省_一般職業紹介状況_有効求人倍率_2025.xlsx
  -> Header found at row index: 1
  -> Date columns identified: Year='西暦', Month='Unnamed: 2'
  -> Extracted rows: 3 (Expected: 12)
  -> Saved: ../data/processed/intermediate/temp_job_ratio.csv
